In [ ]:
!pip install transformers datasets torch peft trl bitsandbytes accelerate

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
model_name="NousResearch/Llama-2-7b-chat-hf"
dataset_name="mlabonne/guanade_llama2_1k"
new_model="llama-2-7b-chat-finetune"
lora_r=64
lora_alpha=16
lora_dropout=0.1
use_4bit=True
bnb_4bit_compute_dtype="float16"
bnb_4bit_quant_type="nf4"
use_nested_quant=False

output_dir="./results"
num_train_epochs=1
fp16=False
bf16=False

per_device_train_batch_size=4
per_device_eval_batch_size=4
gradient_accumulation_steps=1
gradient_checkpointing=True
max_grad_norm=0.3

learning_rate=2e-4
weight_decay=0.001
optim="paged_adamw_32bit"
lr_scheduler_type="cosine"
max_steps=-1
warmup_ratio=0.03

group_by_length=True
save_steps=0
logging_steps=25

max_seq_length=None
packing=False
device_map={"":0}

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"  # or your preferred model
new_model = "llama-2-7b-chat-guanaco"

# QLoRA parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# LoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# TrainingArguments parameters
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = True
packing = True
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 25

# SFT parameters
max_seq_length = 512
packing = False
device_map = {"": 0}

# Load dataset
dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Initialize SFTTrainer with minimal parameters (compatible with older TRL versions)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)

# Set tokenizer after initialization
trainer.tokenizer = tokenizer

# Optional: Set max sequence length if needed
if hasattr(trainer, 'max_seq_length'):
    trainer.max_seq_length = max_seq_length

# Train the model
trainer.train()

# Save the trained model
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

print(f"Model saved to {new_model}")

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.


Step,Training Loss
25,1.347300
50,1.600200
75,1.207500
100,1.425600
125,1.175300
150,1.354600
175,1.171000
200,1.448900
225,1.155100
250,1.516400


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Model saved to llama-2-7b-chat-guanaco
